# Docker & Containers\n\n## Containerization Fundamentals\n\n**What you'll learn:**\n- Container concepts\n- Docker basics (images, containers)\n- Dockerfile creation\n- Docker Compose\n- Multi-stage builds\n- Container orchestration intro\n- Best practices\n\n**Time:** 90-120 minutes

## 1. Why Containers?\n\n### The Problem\n\n**Traditional:**\n```\n\"It works on my machine!\"\n\nDeveloper: Python 3.10, Ubuntu 20.04\nProduction: Python 3.8, CentOS 7\n→ Different behavior, bugs in production\n```\n\n**With Containers:**\n```\nSame image runs everywhere:\n- Developer laptop\n- CI/CD pipeline\n- Production servers\n- Cloud platforms\n```

### Containers vs VMs\n\n```\n┌─────────────────┐   ┌─────────────────┐\n│   Container     │   │       VM        │\n│ ┌─────┬─────┐  │   │  ┌──────────┐   │\n│ │App 1│App 2│  │   │  │   App    │   │\n│ └─────┴─────┘  │   │  ├──────────┤   │\n│ ┌───────────┐  │   │  │Guest OS  │   │\n│ │Container  │  │   │  └──────────┘   │\n│ │ Runtime   │  │   │  ┌──────────┐   │\n│ └───────────┘  │   │  │Hypervisor│   │\n│ ┌───────────┐  │   │  └──────────┘   │\n│ │  Host OS  │  │   │  ┌──────────┐   │\n│ └───────────┘  │   │  │ Host OS  │   │\n└─────────────────┘   └─────────────────┘\n  Lightweight         Resource Heavy\n  Start in ms         Start in minutes\n  MB in size          GB in size\n```

## 2. Docker Basics\n\n### Installation\n\n```bash\n# Ubuntu/Debian\ncurl -fsSL https://get.docker.com | sh\n\n# Add user to docker group (no sudo needed)\nsudo usermod -aG docker $USER\n\n# Verify\ndocker --version\ndocker run hello-world\n```

### Core Concepts\n\n**Image:** Blueprint (like a class)\n**Container:** Running instance (like an object)\n\n```bash\n# Pull image from Docker Hub\ndocker pull python:3.11\n\n# List images\ndocker images\n\n# Run container\ndocker run python:3.11 python --version\n\n# Run interactively\ndocker run -it python:3.11 bash\n\n# Run in background (detached)\ndocker run -d nginx\n\n# List running containers\ndocker ps\n\n# List all containers\ndocker ps -a\n```

## 3. Creating Docker Images\n\n### Dockerfile\n\n**Simple Python app:**

In [ ]:
# Create sample app\n!mkdir -p docker_demo\n!cat > docker_demo/app.py << 'EOF'\nfrom flask import Flask\napp = Flask(__name__)\n\n@app.route('/')\ndef hello():\n    return 'Hello from Docker!'\n\nif __name__ == '__main__':\n    app.run(host='0.0.0.0', port=5000)\nEOF\n\n!cat > docker_demo/requirements.txt << 'EOF'\nFlask==2.3.0\nEOF\n\nprint('✅ Created app files')

In [ ]:
# Create Dockerfile\n!cat > docker_demo/Dockerfile << 'EOF'\n# Base image\nFROM python:3.11-slim\n\n# Set working directory\nWORKDIR /app\n\n# Copy requirements\nCOPY requirements.txt .\n\n# Install dependencies\nRUN pip install --no-cache-dir -r requirements.txt\n\n# Copy application\nCOPY app.py .\n\n# Expose port\nEXPOSE 5000\n\n# Run application\nCMD [\"python\", \"app.py\"]\nEOF\n\nprint('✅ Created Dockerfile')

### Build and Run\n\n```bash\n# Build image\ndocker build -t my-app:1.0 .\n\n# Run container\ndocker run -d -p 5000:5000 --name myapp my-app:1.0\n\n# Test\ncurl http://localhost:5000\n\n# View logs\ndocker logs myapp\n\n# Stop container\ndocker stop myapp\n\n# Remove container\ndocker rm myapp\n```

## 4. Dockerfile Best Practices\n\n### Multi-Stage Builds\n\n```dockerfile\n# Build stage\nFROM node:18 AS builder\nWORKDIR /app\nCOPY package*.json ./\nRUN npm install\nCOPY . .\nRUN npm run build\n\n# Production stage\nFROM nginx:alpine\nCOPY --from=builder /app/dist /usr/share/nginx/html\nEXPOSE 80\nCMD [\"nginx\", \"-g\", \"daemon off;\"]\n```\n\n**Benefits:**\n- Smaller final image\n- No build tools in production\n- More secure

### Layer Caching\n\n```dockerfile\n# ❌ Bad: Cache busted on any code change\nCOPY . .\nRUN pip install -r requirements.txt\n\n# ✅ Good: Dependencies cached separately\nCOPY requirements.txt .\nRUN pip install -r requirements.txt\nCOPY . .\n```

### Security\n\n```dockerfile\n# Don't run as root\nRUN adduser -D myuser\nUSER myuser\n\n# Use specific versions\nFROM python:3.11.4-slim  # Not 'latest'\n\n# Scan for vulnerabilities\n# docker scan my-app:1.0\n```

## 5. Docker Compose\n\n### Multi-Container Apps\n\n**docker-compose.yml:**

In [ ]:
!cat > docker_demo/docker-compose.yml << 'EOF'\nversion: '3.8'\n\nservices:\n  web:\n    build: .\n    ports:\n      - \"5000:5000\"\n    environment:\n      - DATABASE_URL=postgresql://db:5432/myapp\n    depends_on:\n      - db\n  \n  db:\n    image: postgres:15\n    environment:\n      - POSTGRES_PASSWORD=secret\n      - POSTGRES_DB=myapp\n    volumes:\n      - postgres_data:/var/lib/postgresql/data\n  \n  redis:\n    image: redis:7-alpine\n    \nvolumes:\n  postgres_data:\nEOF\n\nprint('✅ Created docker-compose.yml')

### Docker Compose Commands\n\n```bash\n# Start all services\ndocker-compose up -d\n\n# View logs\ndocker-compose logs -f\n\n# Stop services\ndocker-compose down\n\n# Rebuild images\ndocker-compose build\n\n# Scale service\ndocker-compose up -d --scale web=3\n```

## 6. Volumes & Data Persistence\n\n```bash\n# Named volume\ndocker run -v mydata:/app/data myapp\n\n# Bind mount (local directory)\ndocker run -v $(pwd)/data:/app/data myapp\n\n# Read-only\ndocker run -v $(pwd)/config:/app/config:ro myapp\n\n# List volumes\ndocker volume ls\n\n# Inspect volume\ndocker volume inspect mydata\n```

## 7. Networking\n\n```bash\n# Create network\ndocker network create mynetwork\n\n# Run containers in network\ndocker run --network mynetwork --name web myapp\ndocker run --network mynetwork --name db postgres\n\n# Now 'web' can reach 'db' by name!\n# DATABASE_URL=postgresql://db:5432/myapp\n\n# List networks\ndocker network ls\n```

## 8. Container Orchestration\n\n### Kubernetes (K8s)\n\n**For production:**\n- Auto-scaling\n- Self-healing\n- Load balancing\n- Rolling updates\n- Service discovery\n\n**Alternatives:**\n- Docker Swarm (simpler)\n- Amazon ECS\n- Google Cloud Run\n- Azure Container Instances

## 🎯 Exercises\n\n1. Create Dockerfile for Python app\n2. Build and run container\n3. Write docker-compose.yml with database\n4. Implement multi-stage build\n5. Set up volume for persistence\n\n## 🎓 Key Takeaways\n\n- Containers ensure consistency\n- Dockerfile = reproducible builds\n- Multi-stage builds = smaller images\n- Docker Compose = multi-container apps\n- Volumes for data persistence\n- Networks for container communication

## 🚀 Next Steps\n\n1. Learn Kubernetes basics\n2. CI/CD with containers\n3. Security scanning\n4. Container monitoring\n\n## 📚 Resources\n\n- [Docker Docs](https://docs.docker.com/)\n- [Docker Hub](https://hub.docker.com/)\n- [Kubernetes](https://kubernetes.io/)\n- [Play with Docker](https://labs.play-with-docker.com/)